## Install Python Packages

To use Microsoft Agent Framework with Azure OpenAI, install the following Python packages:

```bash
pip install agent-framework --pre python-dotenv
```

In [ ]:
%pip install agent-framework --pre python-dotenv

In [12]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded successfully")

Environment variables loaded successfully


## Create the Agent

First, create a chat client for communicating with Azure OpenAI using the environment variables configured earlier.

Then, create the agent by providing instructions and a name for the agent.

In [8]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

## Run the Agent

To run the agent, call the `run` method on the agent instance, providing the user input. The agent will return a response object, and accessing the `.text` property provides the text result from the agent.

In [10]:
result = await agent.run("Tell me a joke about a pirate.")
print(result.text)

Why did the pirate go to school?

Because he wanted to improve his "arrghticulation!"
